In [0]:
%sql
select current_catalog();

In [0]:
%sql
create catalog if not exists lakeflow_job;

In [0]:
%sql
select current_catalog();

In [0]:
%sql
show catalogs;

In [0]:
%sql
use catalog lakeflow_job;

In [0]:
%sql
create volume if not exists lakeflow_job.default.ranjan_volume;

In [0]:
dbutils.fs.mkdirs('/Volumes/lakeflow_job/default/ranjan_volume/inputdata/customers')

In [0]:

landing_zone = "/Volumes/lakeflow_job/default/ranjan_volume/inputdata"
orders_data =landing_zone+ "/orders"
checkpoint_path = landing_zone + "/inputcheckpoint"

(spark.
 readStream.
 format("cloudFiles")
 .option("cloudFiles.format", "csv")
 .option("cloudFiles.inferColumnTypes", "true")
 .option("cloudFiles.schemaLocation", checkpoint_path)
 .load(orders_data)
 .writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .trigger(availableNow=True)
 .table("lakeflow_job.default.orders_raw")
 )


In [0]:
%sql
--select * from lakeflow_job.default.orders_raw;

drop table lakeflow_job.default.orders_raw;

In [0]:
landing_zone = "/Volumes/lakeflow_job/default/ranjan_volume/inputdata"
customers_data =landing_zone+ "/customers"
checkpoint_path = landing_zone + "/customers_inputcheckpoint"

(spark.
 readStream.
 format("cloudFiles")
 .option("cloudFiles.format", "csv")
 .option("cloudFiles.inferSchema", "true")
 .option("cloudFiles.inferColumnTypes", "true")
 .option("cloudFiles.schemaLocation", checkpoint_path)
 .load(customers_data)
 .writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .trigger(availableNow=True)
 .table("lakeflow_job.default.customers_raw")
 )

In [0]:
%sql
--select * from lakeflow_job.default.customers_raw;
drop table lakeflow_job.default.customers_raw;

In [0]:
%sql
alter table customers_raw
set tblproperties("delta.columnMapping.mode" = "name")


In [0]:
%sql
alter table customers_raw
drop column `_rescued_data`

In [0]:
%sql
--select * from lakeflow_job.bronze.orders_bronze
drop table lakeflow_job.silver.orders_cleaned_silver

In [0]:
%sql
select * from lakeflow_job.silver.orders_cleaned_silver;

In [0]:
%sql
drop table if exists lakeflow_job.bronze.orders_bronze;

In [0]:
%sql
select * from lakeflow_job.silver.orders_cleaned_silver

In [0]:
%sql
desc extended lakeflow_job.silver.orders_cleaned_silver

In [0]:
%sql
select * from lakeflow_job.bronze.orders_bronze;

In [0]:
%sql
select * from lakeflow_job.bronze.customers_bronze

In [0]:
%sql
select * from lakeflow_job.silver.customers_enriched_silver